In [2]:
import tensorflow_hub as hub
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization



model_path = "../Results/model_5_acc_85_loss_41/"

loaded_model = tf.keras.models.load_model(model_path)
                                        

* Split it into sentences (lines).
* split into characters
* find number of each line
* find number of total lines

In [3]:
import pandas as pd
import json

In [5]:
with open("skimlit_example_abstracts.json", "r") as f:
    example_abstracts = json.load(f)

In [6]:
abstracts = pd.DataFrame(example_abstracts)
abstracts

,abstract,source,details
0,This RCT examined the efficacy of a manualized...,https://pubmed.ncbi.nlm.nih.gov/20232240/,RCT of a manualized social treatment for high-...
1,Postpartum depression (PPD) is the most preval...,https://pubmed.ncbi.nlm.nih.gov/28012571/,Formatting removed (can be used to compare mod...
2,"Mental illness, including depression, anxiety ...",https://pubmed.ncbi.nlm.nih.gov/28942748/,Effect of nutrition on mental health
3,Hepatitis C virus (HCV) and alcoholic liver di...,https://pubmed.ncbi.nlm.nih.gov/22244707/,Baclofen promotes alcohol abstinence in alcoho...


In [8]:
import nltk

nltk.download('punkt')  # Download the Punkt tokenizer models

# Example text
example_text = example_abstracts[0]["abstract"]

# Use NLTK for sentence splitting
abstract_lines = nltk.sent_tokenize(example_text)

abstract_lines

[nltk_data] Downloading package punkt to C:\Users\EL AIDI
[nltk_data]     MOHAMED\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


['This RCT examined the efficacy of a manualized social intervention for children with HFASDs.',
 'Participants were randomly assigned to treatment or wait-list conditions.',
 'Treatment included instruction and therapeutic activities targeting social skills, face-emotion recognition, interest expansion, and interpretation of non-literal language.',
 'A response-cost program was applied to reduce problem behaviors and foster skills acquisition.',
 'Significant treatment effects were found for five of seven primary outcome measures (parent ratings and direct child measures).',
 'Secondary measures based on staff ratings (treatment group only) corroborated gains reported by parents.',
 'High levels of parent, child and staff satisfaction were reported, along with high levels of treatment fidelity.',
 'Standardized effect size estimates were primarily in the medium and large ranges and favored the treatment group.']

In [9]:
# Get total number of lines
total_lines_in_sample = len(abstract_lines)

# Go through each line in abstract and create a list of dictionaries containing features for each line
sample_lines = []
for i, line in enumerate(abstract_lines):
    sample_dict = {}
    sample_dict["text"] = str(line)
    sample_dict["line_number"] = i
    sample_dict["total_lines"] = total_lines_in_sample - 1
    sample_lines.append(sample_dict)

sample_lines

[{'text': 'This RCT examined the efficacy of a manualized social intervention for children with HFASDs.',
  'line_number': 0,
  'total_lines': 7},
 {'text': 'Participants were randomly assigned to treatment or wait-list conditions.',
  'line_number': 1,
  'total_lines': 7},
 {'text': 'Treatment included instruction and therapeutic activities targeting social skills, face-emotion recognition, interest expansion, and interpretation of non-literal language.',
  'line_number': 2,
  'total_lines': 7},
 {'text': 'A response-cost program was applied to reduce problem behaviors and foster skills acquisition.',
  'line_number': 3,
  'total_lines': 7},
 {'text': 'Significant treatment effects were found for five of seven primary outcome measures (parent ratings and direct child measures).',
  'line_number': 4,
  'total_lines': 7},
 {'text': 'Secondary measures based on staff ratings (treatment group only) corroborated gains reported by parents.',
  'line_number': 5,
  'total_lines': 7},
 {'text'

In [10]:
# Get all line_number values from sample abstract
test_abstract_line_numbers = [line["line_number"] for line in sample_lines]

# One-hot encode to same depth as training data, so model accepts right input shape
test_abstract_line_numbers_one_hot = tf.one_hot(test_abstract_line_numbers, depth=15) 
test_abstract_line_numbers_one_hot

<tf.Tensor: shape=(8, 15), dtype=float32, numpy=
array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.]],
      dtype=float32)>

In [11]:
# Get all total_lines values from sample abstract
test_abstract_total_lines = [line["total_lines"] for line in sample_lines]
# One-hot encode to same depth as training data, so model accepts right input shape
test_abstract_total_lines_one_hot = tf.one_hot(test_abstract_total_lines, depth=20)
test_abstract_total_lines_one_hot

<tf.Tensor: shape=(8, 20), dtype=float32, numpy=
array([[0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.]], dtype=float32)>

In [12]:
def split_chars(text):
    return ' '.join(list(text))

In [13]:
# Split abstract lines into characters
abstract_chars = [split_chars(sentence) for sentence in abstract_lines]

In [14]:
test_abstract_line_numbers_one_hot

<tf.Tensor: shape=(8, 15), dtype=float32, numpy=
array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.]],
      dtype=float32)>

In [15]:
test_abstract_total_lines_one_hot

<tf.Tensor: shape=(8, 20), dtype=float32, numpy=
array([[0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.]], dtype=float32)>

In [14]:
tf.constant(abstract_lines)

<tf.Tensor: shape=(8,), dtype=string, numpy=
array([b'This RCT examined the efficacy of a manualized social intervention for children with HFASDs.',
       b'Participants were randomly assigned to treatment or wait-list conditions.',
       b'Treatment included instruction and therapeutic activities targeting social skills, face-emotion recognition, interest expansion, and interpretation of non-literal language.',
       b'A response-cost program was applied to reduce problem behaviors and foster skills acquisition.',
       b'Significant treatment effects were found for five of seven primary outcome measures (parent ratings and direct child measures).',
       b'Secondary measures based on staff ratings (treatment group only) corroborated gains reported by parents.',
       b'High levels of parent, child and staff satisfaction were reported, along with high levels of treatment fidelity.',
       b'Standardized effect size estimates were primarily in the medium and large ranges and fav

In [15]:
tf.constant(abstract_chars)

<tf.Tensor: shape=(8,), dtype=string, numpy=
array([b'T h i s   R C T   e x a m i n e d   t h e   e f f i c a c y   o f   a   m a n u a l i z e d   s o c i a l   i n t e r v e n t i o n   f o r   c h i l d r e n   w i t h   H F A S D s .',
       b'P a r t i c i p a n t s   w e r e   r a n d o m l y   a s s i g n e d   t o   t r e a t m e n t   o r   w a i t - l i s t   c o n d i t i o n s .',
       b'T r e a t m e n t   i n c l u d e d   i n s t r u c t i o n   a n d   t h e r a p e u t i c   a c t i v i t i e s   t a r g e t i n g   s o c i a l   s k i l l s ,   f a c e - e m o t i o n   r e c o g n i t i o n ,   i n t e r e s t   e x p a n s i o n ,   a n d   i n t e r p r e t a t i o n   o f   n o n - l i t e r a l   l a n g u a g e .',
       b'A   r e s p o n s e - c o s t   p r o g r a m   w a s   a p p l i e d   t o   r e d u c e   p r o b l e m   b e h a v i o r s   a n d   f o s t e r   s k i l l s   a c q u i s i t i o n .',
       b'S i g n i f i c a n t   t r e a t m e n 

In [16]:
# Make predictions on sample abstract features
test_abstract_pred_probs = loaded_model.predict(x=(test_abstract_line_numbers_one_hot,
                                                   test_abstract_total_lines_one_hot,
                                                   tf.constant(abstract_lines),
                                                   tf.constant(abstract_chars)))
test_abstract_pred_probs

1/1 [==============================] - 2s 2s/step


array([[2.17811748e-01, 2.57731928e-03, 2.65421288e-04, 7.79298306e-01,
        4.71834610e-05],
       [1.89512828e-03, 1.40211205e-05, 9.86449540e-01, 2.19070236e-03,
        9.45061632e-03],
       [1.17194615e-02, 1.83234311e-04, 9.32980299e-01, 1.53515432e-02,
        3.97654399e-02],
       [4.38626632e-02, 5.46142645e-03, 7.01393485e-01, 3.41751352e-02,
        2.15107247e-01],
       [9.11207928e-04, 6.10383134e-03, 1.34519294e-01, 5.85712376e-04,
        8.57879996e-01],
       [7.10648950e-04, 3.31444712e-03, 8.44479799e-01, 3.44797008e-04,
        1.51150331e-01],
       [1.32539251e-04, 5.74295558e-02, 8.33371188e-03, 3.50271876e-05,
        9.34069157e-01],
       [1.99921469e-05, 4.95938547e-02, 3.00278850e-02, 9.75907187e-06,
        9.20348525e-01]], dtype=float32)

In [17]:
# Turn prediction probabilities into prediction classes
test_abstract_preds = tf.argmax(test_abstract_pred_probs, axis=1)
test_abstract_preds
classes = ['BACKGROUND', 'CONCLUSIONS', 'METHODS', 'OBJECTIVE', 'RESULTS']
# Turn prediction class integers into string class names
test_abstract_pred_classes = [classes[i] for i in test_abstract_preds]
test_abstract_pred_classes
# Visualize abstract lines and predicted sequence labels
for i, line in enumerate(abstract_lines):
    print(f"{test_abstract_pred_classes[i]}: {line}")

OBJECTIVE: This RCT examined the efficacy of a manualized social intervention for children with HFASDs.
METHODS: Participants were randomly assigned to treatment or wait-list conditions.
METHODS: Treatment included instruction and therapeutic activities targeting social skills, face-emotion recognition, interest expansion, and interpretation of non-literal language.
METHODS: A response-cost program was applied to reduce problem behaviors and foster skills acquisition.
RESULTS: Significant treatment effects were found for five of seven primary outcome measures (parent ratings and direct child measures).
METHODS: Secondary measures based on staff ratings (treatment group only) corroborated gains reported by parents.
RESULTS: High levels of parent, child and staff satisfaction were reported, along with high levels of treatment fidelity.
RESULTS: Standardized effect size estimates were primarily in the medium and large ranges and favored the treatment group.


In [31]:
import nltk
from nltk.tokenize import sent_tokenize
import tensorflow as tf

def nltk_function(abstract):
    # Tokenize sentences using NLTK
    abstract_lines = sent_tokenize(abstract)
    return abstract_lines

# ---------------------------------------------------------------------------------------------------------------------------

def split_chars(text):
    return ' '.join(list(text))

# ---------------------------------------------------------------------------------------------------------------------------

def make_predictions(text):
    classes = ['BACKGROUND', 'CONCLUSIONS', 'METHODS', 'OBJECTIVE', 'RESULTS']
    abstract_lines = list()

    abstract_lines = nltk_function(text)

    # Get total number of lines
    total_lines_in_sample = len(abstract_lines)

    # Go through each line in abstract and create a list of dictionaries containing features for each line
    sample_lines = []
    for i, line in enumerate(abstract_lines):
        sample_dict = {}
        sample_dict["text"] = str(line)
        sample_dict["line_number"] = i
        sample_dict["total_lines"] = total_lines_in_sample - 1
        sample_lines.append(sample_dict)

    # Get all line_number values from the sample abstract
    test_abstract_line_numbers = [line["line_number"] for line in sample_lines]

    # One-hot encode to the same depth as training data so that the model accepts the right input shape
    test_abstract_line_numbers_one_hot = tf.one_hot(test_abstract_line_numbers, depth=15)

    # Get all total_lines values from the sample abstract
    test_abstract_total_lines = [line["total_lines"] for line in sample_lines]

    # One-hot encode to the same depth as training data so that the model accepts the right input shape
    test_abstract_total_lines_one_hot = tf.one_hot(test_abstract_total_lines, depth=20)

    # Split abstract lines into characters
    abstract_chars = [split_chars(sentence) for sentence in abstract_lines]

    # Make predictions on sample abstract features
    test_abstract_pred_probs = loaded_model.predict(x=(test_abstract_line_numbers_one_hot,
                                                       test_abstract_total_lines_one_hot,
                                                       tf.constant(abstract_lines),
                                                       tf.constant(abstract_chars)))

    test_abstract_preds = tf.argmax(test_abstract_pred_probs, axis=1)

    test_abstract_pred_classes = [classes[i] for i in test_abstract_preds]
    
    
    
    BACKGROUND = ""
    CONCLUSIONS = ""
    METHODS = ""
    OBJECTIVE = ""
    RESULTS = ""

    for i, line in enumerate(abstract_lines):
        print(f"{test_abstract_pred_classes[i]}: {line}")

        current_class = test_abstract_pred_classes[i]

        if current_class == "BACKGROUND":
            BACKGROUND += line
        elif current_class == "CONCLUSIONS":
            CONCLUSIONS += line
        elif current_class == "METHODS":
            METHODS += line
        elif current_class == "OBJECTIVE":
            OBJECTIVE += line
        elif current_class == "RESULTS":
            RESULTS += line
        

    print(BACKGROUND)


In [32]:
# Example abstract text
example_abstract = """
The Darknet is an encrypted corner of the internet,
intended for users who wish to remain anonymous and mask
their identity. Because of its anonymous qualities, the Darknet
has become a go-to platform for illicit activities such as drug
trafficking, terrorism, and dark marketplaces. Therefore, it is
important to recognize Darknet traffic in order to monitor and
detect malicious online activities. This paper investigates the
potential effectiveness of machine learning algorithms in identifying attacks using the CICdarknet2020 dataset. The dataset
includes two distinct classification targets: traffic label and
application labels. The objective of our research is to identify
optimal classifiers for traffic and application classification by
employing ensemble learning methods, aiming to achieve the
highest possible results. Through our experimentation, we have
found that the best-performing models surpassing all other state of-the-art machine learning models are LightGBM, achieving a
93.41% f1-score in the Application classification, and Random
Forest, achieving a 99.8% f1-score in the traffic classification."""

text_predict = make_predictions(example_abstract)
text_predict

1/1 [==============================] - 0s 37ms/step
BACKGROUND: 
The Darknet is an encrypted corner of the internet,
intended for users who wish to remain anonymous and mask
their identity.
BACKGROUND: Because of its anonymous qualities, the Darknet
has become a go-to platform for illicit activities such as drug
trafficking, terrorism, and dark marketplaces.
BACKGROUND: Therefore, it is
important to recognize Darknet traffic in order to monitor and
detect malicious online activities.
OBJECTIVE: This paper investigates the
potential effectiveness of machine learning algorithms in identifying attacks using the CICdarknet2020 dataset.
METHODS: The dataset
includes two distinct classification targets: traffic label and
application labels.
CONCLUSIONS: The objective of our research is to identify
optimal classifiers for traffic and application classification by
employing ensemble learning methods, aiming to achieve the
highest possible results.
CONCLUSIONS: Through our experimentation, we h

In [21]:
def regroup_text(text):
    sections = {
        'BACKGROUND': [],
        'METHODS': [],
        'RESULTS': [],
        'CONCLUSIONS': []
    }

    current_section = None

    for line in text.split('\n'):
        line = line.strip()
        if line in sections:
            current_section = line
        elif current_section:
            sections[current_section].append(line)

    regrouped_text = {}

    for section, content in sections.items():
        regrouped_text[section] = ' '.join(content)

    return regrouped_text

regroup_text(text_predict)

AttributeError: 'NoneType' object has no attribute 'split'